In [ ]:
import pandas as pd

In [2]:
# Step 1: Create base customer data
df_base = pd.DataFrame({
    'CustomerID': [101, 102, 103, 104, 105, 106],
    'Name': ['hasini ganta', 'deepthi belpuri', 'keerthi pk', 'mani reddy', 'vanilla chow', 'poojitha poo'],
    'Address': ['NY', 'CA', 'TX', 'FL', 'CA', 'NV'],
    'Email': ['hasiniganta@gmail.com', 'deepthibelpuri@yahoo.com', 'keerthipk@outlook.com', 'manireddy@msn.com', 'vanillachow@gmail.com', 'poojithapoo@gmail.com'],
    'Phone': ['1234567890', '2345678901', '3456789012', '4567890123', '5678901234', '6789012345']
})

# View the DataFrame
print(df_base)


   CustomerID             Name Address                     Email       Phone
0         101     hasini ganta      NY     hasiniganta@gmail.com  1234567890
1         102  deepthi belpuri      CA  deepthibelpuri@yahoo.com  2345678901
2         103       keerthi pk      TX     keerthipk@outlook.com  3456789012
3         104       mani reddy      FL         manireddy@msn.com  4567890123
4         105     vanilla chow      CA     vanillachow@gmail.com  5678901234
5         106     poojitha poo      NV     poojithapoo@gmail.com  6789012345


In [ ]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.engine import URL
import getpass 
# Step 1: Define connection info
connection_url = URL.create(
    "mssql+pyodbc",
    username="sa",
    password="*******",
    host="DESKTOP-1M0C5L8\\SQLEXPRESS01",
    database="SCD_DEMO",
    query={"driver": "ODBC Driver 17 for SQL Server"}
)

# Step 2: Create engine
engine = create_engine(connection_url)

In [9]:
df_base.to_sql('BASE_TABLE',con=engine,if_exists='replace',index=False)

6

In [ ]:
# scd type-1 overwrite
# Update the record for CustomerID '101'
from sqlalchemy import text 
q1=text('''update BASE_TABLE
set Address='LA',
email='hahahasini@gmail.com'
where CustomerID='101';
''')
with engine.connect() as conn:
    conn.execute(q1)
    conn.commit()
 

In [14]:
import pandas as pd

df_base2 = pd.DataFrame({
    'CustomerID': [201, 202, 203, 204],
    'Name': ['siri kumari', 'bhavya ch', 'nithin sai', 'rahul dravid'],
    'Address': ['AP', 'KA', 'TS', 'TN'],
    'Email': ['siri@gmail.com', 'bhavya@gmail.com', 'nithin@gmail.com', 'rahul@gmail.com'],
    'Phone': ['9000000001', '9000000002', '9000000003', '9000000004'],
    'StartDate': [pd.to_datetime('2024-01-01')] * 4,
    'EndDate': [None] * 4,
    'IsCurrent': [1] * 4
})

print(df_base2)


   CustomerID          Name Address             Email       Phone  StartDate  \
0         201   siri kumari      AP    siri@gmail.com  9000000001 2024-01-01   
1         202     bhavya ch      KA  bhavya@gmail.com  9000000002 2024-01-01   
2         203    nithin sai      TS  nithin@gmail.com  9000000003 2024-01-01   
3         204  rahul dravid      TN   rahul@gmail.com  9000000004 2024-01-01   

  EndDate  IsCurrent  
0    None          1  
1    None          1  
2    None          1  
3    None          1  


In [15]:
# Upload base data to SQL Server
df_base2.to_sql('BASE_TABLE2', con=engine, if_exists='replace', index=False)

4

In [16]:
# Simulated incoming data (a few updates for CustomerID 202 and 204)
df_changes = pd.DataFrame({
    'CustomerID': [202, 204],
    'Name': ['bhavya ch', 'rahul dravid'],
    'Address': ['MH', 'KL'],  # changed from KA and TN
    'Email': ['bhavya@gmail.com', 'rahul@gmail.com'],  # same
    'Phone': ['9111111111', '9222222222'],  # changed
    'StartDate': [pd.to_datetime('2025-06-22')] * 2,  # today
    'EndDate': [None] * 2,
    'IsCurrent': [1] * 2
})

print(df_changes)


   CustomerID          Name Address             Email       Phone  StartDate  \
0         202     bhavya ch      MH  bhavya@gmail.com  9111111111 2025-06-22   
1         204  rahul dravid      KL   rahul@gmail.com  9222222222 2025-06-22   

  EndDate  IsCurrent  
0    None          1  
1    None          1  


In [17]:
from sqlalchemy import text
from datetime import datetime

today = datetime.today().strftime('%Y-%m-%d')

with engine.begin() as conn:
    for _, row in df_changes.iterrows():
        # Step 1: Expire the old row
        conn.execute(text('''
            UPDATE BASE_TABLE2
            SET EndDate = :today, IsCurrent = 0
            WHERE CustomerID = :cid AND IsCurrent = 1
        '''), {'today': today, 'cid': row['CustomerID']})

        # Step 2: Insert new version
        conn.execute(text('''
            INSERT INTO BASE_TABLE2 (
                CustomerID, Name, Address, Email, Phone, StartDate, EndDate, IsCurrent
            ) VALUES (
                :cid, :name, :addr, :email, :phone, :start, NULL, 1
            )
        '''), {
            'cid': row['CustomerID'],
            'name': row['Name'],
            'addr': row['Address'],
            'email': row['Email'],
            'phone': row['Phone'],
            'start': today
        })


In [18]:
df_updated = pd.read_sql("SELECT * FROM BASE_TABLE2", con=engine)
print(df_updated)
# Step 3: Verify the updates
print("Updated BASE_TABLE2:")   


   CustomerID          Name Address             Email       Phone  StartDate  \
0         201   siri kumari      AP    siri@gmail.com  9000000001 2024-01-01   
1         202     bhavya ch      KA  bhavya@gmail.com  9000000002 2024-01-01   
2         203    nithin sai      TS  nithin@gmail.com  9000000003 2024-01-01   
3         204  rahul dravid      TN   rahul@gmail.com  9000000004 2024-01-01   
4         202     bhavya ch      MH  bhavya@gmail.com  9111111111 2025-06-23   
5         204  rahul dravid      KL   rahul@gmail.com  9222222222 2025-06-23   

      EndDate  IsCurrent  
0        None          1  
1  2025-06-23          0  
2        None          1  
3  2025-06-23          0  
4        None          1  
5        None          1  
Updated BASE_TABLE2:


In [20]:
import pandas as pd

df_base3 = pd.DataFrame({
    'CustomerID': [201, 202, 203, 204, 202, 204],
    'Name': ['siri kumari', 'bhavya ch', 'nithin sai', 'rahul dravid', 'bhavya ch', 'rahul dravid'],
    'Address': ['AP', 'KA', 'TS', 'TN', 'MH', 'KL'],
    'Email': ['siri@gmail.com', 'bhavya@gmail.com', 'nithin@gmail.com', 'rahul@gmail.com',
              'bhavya@gmail.com', 'rahul@gmail.com'],
    'Phone': ['9000000001', '9000000002', '9000000003', '9000000004', '9111111111', '9222222222'],
    'StartDate': pd.to_datetime(['2024-01-01', '2024-01-01', '2024-01-01', '2024-01-01',
                                 '2025-06-23', '2025-06-23']),
    'EndDate': [None, '2025-06-23', None, '2025-06-23', None, None],
    'IsCurrent': [1, 0, 1, 0, 1, 1]
})

print(df_base3)


   CustomerID          Name Address             Email       Phone  StartDate  \
0         201   siri kumari      AP    siri@gmail.com  9000000001 2024-01-01   
1         202     bhavya ch      KA  bhavya@gmail.com  9000000002 2024-01-01   
2         203    nithin sai      TS  nithin@gmail.com  9000000003 2024-01-01   
3         204  rahul dravid      TN   rahul@gmail.com  9000000004 2024-01-01   
4         202     bhavya ch      MH  bhavya@gmail.com  9111111111 2025-06-23   
5         204  rahul dravid      KL   rahul@gmail.com  9222222222 2025-06-23   

      EndDate  IsCurrent  
0        None          1  
1  2025-06-23          0  
2        None          1  
3  2025-06-23          0  
4        None          1  
5        None          1  


In [21]:
df_base3.to_sql("BASE_TABLE3", con=engine, if_exists="replace", index=False)


6

In [24]:
from sqlalchemy import text

with engine.begin() as conn:
    conn.execute(text('''
        ALTER TABLE BASE_TABLE3
        ADD Previous_Address VARCHAR(100);
    '''))


In [25]:
df_changes3 = pd.DataFrame({
    'CustomerID': [203],
    'Address': ['GJ']  # or Email 
})  

In [27]:
import sqlalchemy
from sqlalchemy import text
with engine.begin() as conn:
    for _, row in df_changes3.iterrows():
        conn.execute(text('''
            UPDATE BASE_TABLE3
            SET 
                Previous_Address = Address,
                Address = :new_addr
            WHERE CustomerID = :cid AND IsCurrent = 1
        '''), {
            'new_addr': row['Address'],
            'cid': row['CustomerID']
        })


In [28]:
df_type3 = pd.read_sql("SELECT * FROM BASE_TABLE3", con=engine)
print(df_type3)

   CustomerID          Name Address             Email       Phone  StartDate  \
0         201   siri kumari      AP    siri@gmail.com  9000000001 2024-01-01   
1         202     bhavya ch      KA  bhavya@gmail.com  9000000002 2024-01-01   
2         203    nithin sai      GJ  nithin@gmail.com  9000000003 2024-01-01   
3         204  rahul dravid      TN   rahul@gmail.com  9000000004 2024-01-01   
4         202     bhavya ch      MH  bhavya@gmail.com  9111111111 2025-06-23   
5         204  rahul dravid      KL   rahul@gmail.com  9222222222 2025-06-23   

      EndDate  IsCurrent Previous_Address  
0        None          1             None  
1  2025-06-23          0             None  
2        None          1               TS  
3  2025-06-23          0             None  
4        None          1             None  
5        None          1             None  


In [29]:
# create a new dataframe for SCD Type 3,starting fresh again
import pandas as pd

df_base3 = pd.DataFrame({
    'CustomerID': [101, 102, 103, 104, 105, 106],
    'Name': ['hasini ganta', 'deepthi belpuri', 'keerthi pk', 'mani reddy', 'vanilla chow', 'poojitha poo'],
    'Address': ['LA', 'CA', 'TX', 'FL', 'CA', 'NV'],
    'Email': ['hahahasini@gmail.com', 'deepthibelpuri@yahoo.com', 'keerthipk@outlook.com', 'manireddy@msn.com',
              'vanillachow@gmail.com', 'poojithapoo@gmail.com'],
    'Phone': ['1234567890', '2345678901', '3456789012', '4567890123', '5678901234', '6789012345']
})

# Load to SQL Server
df_base3.to_sql("BASE_TABLE_3", con=engine, if_exists="replace", index=False)


6

In [31]:
from sqlalchemy import text

with engine.begin() as conn:
    conn.execute(text("ALTER TABLE BASE_TABLE_3 ADD Previous_Address VARCHAR(100)"))


In [32]:
df_changes3 = pd.DataFrame({
    'CustomerID': [101],
    'Address': ['NY']
})
with engine.begin() as conn:
    for _, row in df_changes3.iterrows():
        conn.execute(text('''
            UPDATE BASE_TABLE_3
            SET 
                Previous_Address = Address,
                Address = :new_addr
            WHERE CustomerID = :cid 
        '''), {
            'new_addr': row['Address'],
            'cid': row['CustomerID']
        }) 


In [33]:
df_result = pd.read_sql("SELECT * FROM BASE_TABLE_3", con=engine)
print(df_result)


   CustomerID             Name Address                     Email       Phone  \
0         101     hasini ganta      NY      hahahasini@gmail.com  1234567890   
1         102  deepthi belpuri      CA  deepthibelpuri@yahoo.com  2345678901   
2         103       keerthi pk      TX     keerthipk@outlook.com  3456789012   
3         104       mani reddy      FL         manireddy@msn.com  4567890123   
4         105     vanilla chow      CA     vanillachow@gmail.com  5678901234   
5         106     poojitha poo      NV     poojithapoo@gmail.com  6789012345   

  Previous_Address  
0               LA  
1             None  
2             None  
3             None  
4             None  
5             None  


In [35]:
#Add new columns for tracking previous values
#add Previous_Email and Previous_Phone:

from sqlalchemy import text

with engine.begin() as conn:
    conn.execute(text("ALTER TABLE BASE_TABLE_3 ADD Previous_Email VARCHAR(100)"))
    conn.execute(text("ALTER TABLE BASE_TABLE_3 ADD Previous_Phone VARCHAR(100)"))




In [36]:
df_email_change = pd.DataFrame({
    'CustomerID': [103],
    'Email': ['keerthi123@gmail.com']
})

df_phone_change = pd.DataFrame({
    'CustomerID': [106],
    'Phone': ['9876543210']
})
with engine.begin() as conn:
    for _, row in df_email_change.iterrows():
        conn.execute(text('''
            UPDATE BASE_TABLE_3
            SET 
                Previous_Email = Email,
                Email = :new_email
            WHERE CustomerID = :cid 
        '''), {
            'new_email': row['Email'],
            'cid': row['CustomerID']
        })

    for _, row in df_phone_change.iterrows():
        conn.execute(text('''
            UPDATE BASE_TABLE_3
            SET 
                Previous_Phone = Phone,
                Phone = :new_phone
            WHERE CustomerID = :cid 
        '''), {
            'new_phone': row['Phone'],
            'cid': row['CustomerID']
        })

In [37]:
df_final = pd.read_sql("SELECT * FROM BASE_TABLE_3", con=engine)
print(df_final)


   CustomerID             Name Address                     Email       Phone  \
0         101     hasini ganta      NY      hahahasini@gmail.com  1234567890   
1         102  deepthi belpuri      CA  deepthibelpuri@yahoo.com  2345678901   
2         103       keerthi pk      TX      keerthi123@gmail.com  3456789012   
3         104       mani reddy      FL         manireddy@msn.com  4567890123   
4         105     vanilla chow      CA     vanillachow@gmail.com  5678901234   
5         106     poojitha poo      NV     poojithapoo@gmail.com  9876543210   

  Previous_Address         Previous_Email Previous_Phone  
0               LA                   None           None  
1             None                   None           None  
2             None  keerthipk@outlook.com           None  
3             None                   None           None  
4             None                   None           None  
5             None                   None     6789012345  


In [38]:
#scd type 4
df_base4 = pd.DataFrame({
    'CustomerID': [201, 202, 203, 204],
    'Name': ['siri kumari', 'bhavya ch', 'nithin sai', 'rahul dravid'],
    'Address': ['AP', 'KA', 'TS', 'TN'],
    'Email': ['siri@gmail.com', 'bhavya@gmail.com', 'nithin@gmail.com', 'rahul@gmail.com'],
    'Phone': ['9000000001', '9000000002', '9000000003', '9000000004'],
    'LastUpdated': pd.to_datetime(['2024-01-01'] * 4)
})

# Load into SQL Server as BASE_TABLE_4
df_base4.to_sql("BASE_TABLE_4", con=engine, if_exists="replace", index=False)


4

In [40]:
from sqlalchemy import text

with engine.begin() as conn:
    conn.execute(text('''
        CREATE TABLE  HISTORY_TABLE_4 (
            CustomerID INT,
            Name VARCHAR(100),
            Address VARCHAR(100),
            Email VARCHAR(100),
            Phone VARCHAR(20),
            LastUpdated DATETIME
        )
    '''))


In [41]:
df_changes4 = pd.DataFrame({
    'CustomerID': [202, 205],
    'Name': ['bhavya ch', 'nanda sai'],
    'Address': ['MH', 'OD'],
    'Email': ['bhavya@gmail.com', 'nandasai@gmail.com'],
    'Phone': ['9111111111', '9333333333'],
    'LastUpdated': [pd.to_datetime('2025-06-23'), pd.to_datetime('2025-06-23')]
})


In [42]:
with engine.begin() as conn:
    for _, row in df_changes4.iterrows():
        # Check if the customer exists in BASE_TABLE_4
        result = conn.execute(
            text("SELECT COUNT(*) FROM BASE_TABLE_4 WHERE CustomerID = :cid"),
            {"cid": row['CustomerID']}
        ).scalar()
        
        if result > 0:
            # Existing customer → store history and update base
            conn.execute(text('''
                INSERT INTO HISTORY_TABLE_4 (CustomerID, Name, Address, Email, Phone, LastUpdated)
                SELECT CustomerID, Name, Address, Email, Phone, LastUpdated
                FROM BASE_TABLE_4
                WHERE CustomerID = :cid
            '''), {'cid': row['CustomerID']})

            conn.execute(text('''
                UPDATE BASE_TABLE_4
                SET Name = :name,
                    Address = :addr,
                    Email = :email,
                    Phone = :ph,
                    LastUpdated = :upd
                WHERE CustomerID = :cid
            '''), {
                'cid': row['CustomerID'],
                'name': row['Name'],
                'addr': row['Address'],
                'email': row['Email'],
                'ph': row['Phone'],
                'upd': row['LastUpdated']
            })
        else:
            # New customer → insert directly into BASE_TABLE_4
            conn.execute(text('''
                INSERT INTO BASE_TABLE_4 (CustomerID, Name, Address, Email, Phone, LastUpdated)
                VALUES (:cid, :name, :addr, :email, :ph, :upd)
            '''), {
                'cid': row['CustomerID'],
                'name': row['Name'],
                'addr': row['Address'],
                'email': row['Email'],
                'ph': row['Phone'],
                'upd': row['LastUpdated']
            })


In [44]:
df_base = pd.read_sql("SELECT * FROM BASE_TABLE_4", con=engine)

print(df_base)

   CustomerID          Name Address               Email       Phone  \
0         201   siri kumari      AP      siri@gmail.com  9000000001   
1         202     bhavya ch      MH    bhavya@gmail.com  9111111111   
2         203    nithin sai      TS    nithin@gmail.com  9000000003   
3         204  rahul dravid      TN     rahul@gmail.com  9000000004   
4         205     nanda sai      OD  nandasai@gmail.com  9333333333   

  LastUpdated  
0  2024-01-01  
1  2025-06-23  
2  2024-01-01  
3  2024-01-01  
4  2025-06-23  


In [45]:
# View history table (only old versions of changed records)
df_history = pd.read_sql("SELECT * FROM HISTORY_TABLE_4", con=engine)

print(df_history)

   CustomerID       Name Address             Email       Phone LastUpdated
0         202  bhavya ch      KA  bhavya@gmail.com  9000000002  2024-01-01
